In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(""),"../python")))

import pandas as pd
import numpy as np

import data_utils as du
import lacp_utils as lu

CENSOR_DATE = pd.to_datetime('2022-11-28')


In [2]:
df1 = du.get_dbs_new()                                       # Data from DBS
df2 = du.get_pdis()                                          # Data from PDIS
df3 = pd.read_csv("../../data/dbs_new_pdis_map.csv")         # DBS to PDIS mapping
df4 = pd.read_csv("../../data/affordable_list.csv")          # Affordability data
df5 = pd.read_csv("../../data/ladwp_project_indicators.csv") # LADWP data

In [3]:
# Merge DBS permit numbers to planning case numbers
# Resulting dataset is one row per PD case

merge_cols_right = [col for col in list(df2.columns) if col not in list(df1.columns)]

dfl = df1.merge(df3, on='PERMIT_NBR', how='left')
dfl = dfl.merge(df2[merge_cols_right], on='CASE_NUMBER', how='left')

dfl['TIME_TO_COMPLETION'] = (dfl['COMPLETION_DATE'] - dfl['FILING_DATE'])/pd.to_timedelta(1,unit='D')   # Calculate case time to completion


In [4]:
# Create wide format data
# Resulting dataset is one row per project

dfw = dfl.groupby(['PERMIT_NBR', 'SUBMITTED_DATE', 'ISSUE_DATE', 'COFO_DATE', 'DU_CHANGED'], dropna=False).agg(
    FIRST_CASE_FILING_DATE = ('FILING_DATE', 'min'), 
    N_CASES = ('CASE_NUMBER', 'nunique')
).reset_index()


In [5]:
# Calculate start dates and development time

dfw['START_DATE'] = dfw[['SUBMITTED_DATE', 'FIRST_CASE_FILING_DATE']].min(axis=1)
dfw['TOTAL_TIME'] = (dfw['COFO_DATE'] - dfw['START_DATE'])/pd.to_timedelta(1,unit='D')

dfw['CENSORED_TIME'] = dfw['TOTAL_TIME']
dfw.loc[ dfw['COFO_DATE'].isnull(), 'CENSORED_TIME'] = (CENSOR_DATE - dfw.loc[ dfw['COFO_DATE'].isnull(), 'START_DATE'])/pd.to_timedelta(1,unit='D')

dfw['IS_COMPLETE'] = (dfw['COFO_DATE'].notnull())*1

dfw['PLANCHECK_TIME'] = (dfw['ISSUE_DATE'] - dfw['SUBMITTED_DATE'])/pd.to_timedelta(1,unit='D')
dfw['APPROVAL_TIME'] = (dfw['ISSUE_DATE'] - dfw['START_DATE'])/pd.to_timedelta(1,unit='D')
dfw['CONSTRUCTION_TIME'] = (dfw['COFO_DATE'] - dfw['ISSUE_DATE'])/pd.to_timedelta(1,unit='D')


In [6]:
# Merge the rest of the DBS data onto dfw

ignore_cols = ['SUBMITTED_DATE','ISSUE_DATE','COFO_DATE','DU_CHANGED']
r_cols = [col for col in df1.columns if col not in ignore_cols]
dfw = dfw.merge(df1[r_cols], on='PERMIT_NBR', how='left')


In [7]:
# Merge affordability information

dfw = dfw.merge(df4[['PERMIT_NBR','AFFORDABLE_UNITS']], on='PERMIT_NBR', how='left')
dfw['AFFORDABLE_UNITS'] = dfw['AFFORDABLE_UNITS'].fillna(0)
dfw['MARKETRATE_UNITS'] = dfw['DU_CHANGED'] - dfw['AFFORDABLE_UNITS']

dfw['AFFORDABLE'] = dfw['AFFORDABLE_UNITS'] / dfw['DU_CHANGED'] >= 0.5
dfw['MIXEDINCOME'] = (dfw['AFFORDABLE_UNITS']>0) & (dfw['AFFORDABLE_UNITS']/dfw['DU_CHANGED']<0.5)


In [8]:
# Merge LADWP information

dfw = dfw.merge(df5, on='PERMIT_NBR', how='left')
dfw['POWER_OH'] = dfw['POWER_OH'].fillna(0)
dfw['POWER_UG'] = dfw['POWER_UG'].fillna(0)
dfw['POWER_CONDUIT'] = dfw['POWER_CONDUIT'].fillna(0)


In [9]:
# Drop permits that expired or are closed

dfw = dfw.loc[ dfw['STATUS'].isin(['CofO Issued', 'Issued', 'CofO Corrected', 'Permit Finaled', 'CofO in Progress']) ].reset_index(drop=True)


In [10]:
# Attach prefix and suffix dummies

dfl['CASE_NUMBER'].fillna('',inplace=True)
dfl = lu.attach_prefix_dummies(dfl, 'CASE_NUMBER')
dfl = lu.attach_suffix_dummies(dfl, 'CASE_NUMBER')

# Collapse the dummies to max 
aggs = {}
for pfx in lu.PREFIX_DUMMIES:
    aggs[pfx] = 'max'
for sfx in lu.SUFFIX_DUMMIES:
    aggs[sfx] = 'max'
df_temp = dfl.groupby('PERMIT_NBR').agg(aggs).reset_index()

dfw = dfw.merge(df_temp, on='PERMIT_NBR', how='left')


In [11]:
# Export the data

dfw.to_pickle("../../intermediate/data_main.pkl")
dfw.to_csv("../../intermediate/data_main.csv", header=True, index=False)
